<a href="https://colab.research.google.com/github/MuthomiTed/EDA-with-SQL/blob/master/Independent_Project_2019_06_Ted_Muthomi_SQL_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="black"> SQL INDEPENDENT PROJECT

## <font color = "purple"> Connecting to SQL Database



In [0]:
# We shall first load the SQL extension into our environment;

%load_ext sql


# Then connect to our in memory sqlite database.

%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

## <font color= "purple"> Importing  Data from a CSV  File

In [0]:
# Importing the python libraries we will need in this notebook;

import csv
import pandas as pd

In [0]:
# Loading the first table from a local csv file;

with open ("GrandElectors_by_state.csv", "r") as f:
  Grandelectors = pd.read_csv (f, index_col = 0, encoding = "utf-8")
  


In [0]:
# Then storing it in an SQL table of our in memory sqlite database

%sql DROP TABLE IF EXISTS Grandelectors 
%sql PERSIST Grandelectors
%sql select * from Grandelectors limit 3;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11


In [0]:
# loading the second table from a local csv file

with open ("Population_by_state.csv", "r") as f:
  Population = pd.read_csv (f, index_col = 0, encoding = "utf-8")

In [0]:
# Then storing it in an SQL table of our in memory sqlite database

%sql DROP TABLE IF EXISTS Population
%sql PERSIST Population
%sql select * from Population limit 3;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272




---



## <font color = "green"> Project  Overview

The aim of this project is to aid a U.S.A. Presidential candidate to become US president.

The winner of the election will be the candidate winning the most grand electors. 

Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win 
 (roughly, but not exactly, proportional to the size of the state)  and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.
 
Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. 

For example, if we assume here there are only 2 candidates,  and no history (no trend of certain states to vote for a particular candidate or party). 
Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.

We are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.

We are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm 
(brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive).

It is ok not to find the overall optimum, we just want a strategy that makes sense.

(This is of course a very simplistic view of reality, but it is a nice starting point to play with data and analyze possibilities).

First we take a few minutes to think about what we need to do.

Steps will be suggested hereafter,  but a big part of the data scientist's job is to identify the flow of operations when being asked a practical question, 
so it is important we start exercising on that in addition to programming.

Here is what we are suggesting to do: we will rank states by decreasing number of grand electors per capita. 
The first states in the list will be the most valuable (we get a large number of grand electors by convincing a small number of people to vote for you). 

We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.

To do that, we need (we are allowed to create as many intermediary tables as you want, to keep queries short):

     1. To join the 2 tables:
              We notice States are not capitalized the same way in both tables 
              (one is in uppercase letters, the other not), so we will first need to convert all to uppercase, for instance.
              Now we can join the tables on the state key.

      2. Our boss wants us to change the name of the "District of Columbia" state to its short version "DC". 

      3. To compute the ratio between the number of grand electors and the population in a new column with that ratio.

      4. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.

      5. To compute the running total of Grand Electors in that sorted list.          

     6. Independently, to compute the half of the total of Grand Electors overall (in the whole country):
              This is the threshold we need to reach for winning the presidential election.

      7. To filter our sorted list of states in order to keep only the (top) ones 
         enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.
                Hint: We can do that in 2 steps:
                     a. Selecting all the states for which the running total is below or equal to the threshold.
                     b. Adding the first state for which the running total is larger than the threshold.



---





---



##  <font color="brown"> Challenge 1: To join the 2 tables;
 States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so we will first need to convert all to uppercase, for instance. Then we can join the tables on the state key.


In [0]:
# We shall first convert both tables to uppercase

%%sql
CREATE TABLE Elections AS
SELECT upper(Population.State) as "STATES", upper(Grandelectors.State) as "STATES_1", upper(Population.State) as "STATES_2",  Population, GrandElectors as "Grandelectors",Grandelectors.State as "STATES_GE_TABLE", Population.State as "STATES_POP_TABLE"
FROM   Grandelectors INNER JOIN Population 
ON     upper(Grandelectors.State) = upper(Population.State) ;

SELECT  * 
from    Elections
LIMIT   5

 * sqlite://
Done.
Done.


STATES,STATES_1,STATES_2,Population,Grandelectors,STATES_GE_TABLE,STATES_POP_TABLE
ALABAMA,ALABAMA,ALABAMA,4858979,9,Alabama,ALABAMA
ALASKA,ALASKA,ALASKA,738432,3,Alaska,ALASKA
ARIZONA,ARIZONA,ARIZONA,6828065,11,Arizona,ARIZONA
ARKANSAS,ARKANSAS,ARKANSAS,2978204,6,Arkansas,ARKANSAS
CALIFORNIA,CALIFORNIA,CALIFORNIA,39144818,55,California,CALIFORNIA


## <font color = "brown"> Challenge 2: To change the name of the "District of Columbia" state to its short version "DC".

In [0]:
%%sql
UPDATE Elections
SET    STATES = "DC" ,STATES_1 = "DC", STATES_2 = "DC"
WHERE  POPULATION = 672228;

SELECT *
FROM   Elections
WHERE  STATES = "DC"

 * sqlite://
1 rows affected.
Done.


STATES,STATES_1,STATES_2,Population,Grandelectors,STATES_GE_TABLE,STATES_POP_TABLE
DC,DC,DC,672228,3,District of Columbia,DISTRICT OF COLUMBIA


## <font color = "brown"> Challenge 3: To compute the ratio between the number of grand electors and the population.

In [0]:
%%sql
SELECT STATES, Population, Grandelectors, Population / Grandelectors as "Population_repped_by_one_GE", ( (Population / Grandelectors * 100.0 ) / Population) as "Pop_%_repped by one_Grandelector"
FROM   Elections
LIMIT  5;


 * sqlite://
Done.


STATES,Population,Grandelectors,Population_repped_by_one_GE,Pop_%_repped by one_Grandelector
ALABAMA,4858979,9,539886,11.111099677524846
ALASKA,738432,3,246144,33.333333333333336
ARIZONA,6828065,11,620733,9.090906428102251
ARKANSAS,2978204,6,496367,16.666655474238837
CALIFORNIA,39144818,55,711723,1.8181793564604132


## <font color = "brown"> Challenge 4: To order the states by decreasing ratio of Grand Electors per capita.

In [0]:
%%sql
SELECT    STATES, Population, Grandelectors, Population / Grandelectors as "Population_repped_by_one_GE", ( (Population / Grandelectors * 100.0 ) / Population) as "Pop_%_repped by one_Grandelector"
FROM      Elections
ORDER BY  Grandelectors
desc
LIMIT     5;

 * sqlite://
Done.


STATES,Population,Grandelectors,Population_repped_by_one_GE,Pop_%_repped by one_Grandelector
CALIFORNIA,39144818,55,711723,1.8181793564604132
TEXAS,27469114,38,722871,2.6315774145463884
FLORIDA,20271272,29,699009,3.4482739908970683
NEW YORK,19795791,29,682613,3.4482734233757064
ILLINOIS,12859995,20,642999,4.999994167960407


## <font color = "brown"> Challenge 5: To compute the running total of Grand Electors in that sorted list.

In [0]:
%%sql
SELECT  A.STATES, A.Population, A.Grandelectors, SUM(B.Grandelectors) as "Running total", SUM(B.Population)
FROM Elections as A
INNER JOIN Elections as B ON (A.Population >= B.Population)
GROUP BY A.Population, A.Grandelectors
ORDER BY SUM(B.Grandelectors) 



 * sqlite://
Done.


STATES,Population,Grandelectors,Running total,SUM(B.Population)
WYOMING,586107,3,3,586107
VERMONT,626042,3,6,1212149
DC,672228,3,9,1884377
ALASKA,738432,3,12,2622809
NORTH DAKOTA,756927,3,15,3379736
SOUTH DAKOTA,858469,3,18,4238205
DELAWARE,945934,3,21,5184139
MONTANA,1032949,3,24,6217088
RHODE ISLAND,1056298,4,28,7273386
MAINE,1329328,4,32,8602714


## <FONT COLOR = "brown"> Challenge 6: Independently, to compute the half of the total of Grand Electors overall (in the whole country).

In [0]:
%%sql
SELECT   (SUM(Grandelectors) / 2 )
FROM     Elections


 * sqlite://
Done.


(SUM(Grandelectors) / 2 )
269


## <font color = "brown"> Challenge 7: To filter our sorted list of states in order to keep only the (top) ones.

In [0]:
%%sql
SELECT  A.STATES, A.Population, A.Grandelectors, SUM(B.Grandelectors) as "Running total", SUM(B.Population)
FROM Elections as A
INNER JOIN Elections as B ON (A.Population >= B.Population)
GROUP BY A.Population, A.Grandelectors
ORDER BY SUM(B.Grandelectors) 
limit 41


 * sqlite://
Done.


STATES,Population,Grandelectors,Running total,SUM(B.Population)
WYOMING,586107,3,3,586107
VERMONT,626042,3,6,1212149
DC,672228,3,9,1884377
ALASKA,738432,3,12,2622809
NORTH DAKOTA,756927,3,15,3379736
SOUTH DAKOTA,858469,3,18,4238205
DELAWARE,945934,3,21,5184139
MONTANA,1032949,3,24,6217088
RHODE ISLAND,1056298,4,28,7273386
MAINE,1329328,4,32,8602714


In [0]:
# OPTION II
%%sql
SELECT  A.STATES, A.Population, A.Grandelectors, SUM(B.Grandelectors) as "Running total", SUM(B.Population)
FROM Elections as A
INNER JOIN Elections as B ON (A.Population <= B.Population)
GROUP BY A.Population, A.Grandelectors
ORDER BY SUM(B.Grandelectors) 
LIMIT 11


 * sqlite://
Done.


STATES,Population,Grandelectors,Running total,SUM(B.Population)
CALIFORNIA,39144818,55,55,39144818
TEXAS,27469114,38,93,66613932
FLORIDA,20271272,29,122,86885204
NEW YORK,19795791,29,151,106680995
ILLINOIS,12859995,20,171,119540990
PENNSYLVANIA,12802503,20,191,132343493
OHIO,11613423,18,209,143956916
GEORGIA,10214860,16,225,154171776
NORTH CAROLINA,10042802,15,240,164214578
MICHIGAN,9922576,16,256,174137154
